# Doc2vec approach (12th December 2021)

TThis notebook applies word2vec to the corpus of tribunal decisions.

In particular, the notebook does:

1. Data preparation for word2vec.

2. Implementation of word2vec averaged per document.

3. Implementation of doc2vec.

This notebook should run in the tfm environment, which can be created with the environment.yml file.

In [1]:
import ipykernel
from os import listdir
from os.path import isfile, join, getsize
import numpy as np
import time
import re
import json
import pickle
import pandas as pd
import sys
import datetime
from tqdm import tqdm

from gensim.models import Word2Vec
import multiprocessing
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score


from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import remove_stopwords


import sys
IN_COLAB = 'google.colab' in sys.modules


# What environment am I using?
print(f'Current environment: {sys.executable}')

# Change the current working directory
os.chdir('/Users/albertamurgopacheco/Documents/GitHub/TFM')
# What's my working directory?
print(f'Current working directory: {os.getcwd()}')


Current environment: /Users/albertamurgopacheco/anaconda3/envs/tfm/bin/python
Current working directory: /Users/albertamurgopacheco/Documents/GitHub/TFM


In [2]:
# Define working directories in colab and local execution

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    docs_path = '/content/gdrive/MyDrive/TFM/data/raw'
    input_path = '/content/gdrive/MyDrive/TFM'
    output_path = '/content/gdrive/MyDrive/TFM/output'

else:
    docs_path = './data/raw'
    input_path = '.'
    output_path = './output'

# DOC2VEC: PARTIAL & FULL TEXT OF THE RULING

# 1. The data and functions needed for the averaged word2vec and the doc2vec

Removing two corrupt files from the corpus.



In [3]:
# Open jsonDataFinal file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# List to store the judicial decisions
corpus = []

corruptFiles = ['HU077022015', 'HU029682017']

# Search data list of dictionaries for dict where {"File":} = file_name
for d in tqdm(data):
    # Dealing with corrupt and empty files
    if d.get('File') not in corruptFiles:
        doc = d.get('String')
        #dec = d.get('Decision:')
        #dec_label = d.get('Decision label:')
        if doc:
            corpus.append(doc)
            #decisions.append(dec)
            #decisions_labels.append(dec_label)
        else:
            continue

print(f'The corpus includes {len(corpus)} documents.')
#print(f'The number of documents with a decision: {len(decisions)}.')
#print(f'The number of decisions with a label: {len(decisions_labels)} documents.')

print(f'The documents are of type: {type(corpus[0])}.')

100%|██████████| 35305/35305 [00:00<00:00, 224197.04it/s]

The corpus includes 35305 documents.
The documents are of type: <class 'str'>.


The text of the decisions (and the labels) is clean. It was cleaned during the information extraction process because the exctraction required tokenization (stanza). The text of the strategies (corpus) needs to be cleaned.

In [4]:
# Gensim-implemented filters for preprocessing data
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, 
strip_multiple_whitespaces, strip_non_alphanum, strip_numeric, remove_stopwords]

# List storing the preprocessed documents
corpus_clean = [preprocess_string(doc, CUSTOM_FILTERS) for doc in corpus]


Some functions:

In [6]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding
    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features


def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

# 2. AVERAGED WORD2VEC ON THE FULL TEXT OF THE RULING

The strategy consists on training a word2vec model using the entire text of all the 35305 rulings. The results are averaged per document. Clustering algorithms are used to identify types of documents and their topics. 

In [29]:
# Number of processing cores
cores = multiprocessing.cpu_count()

w2v_model = Word2Vec(min_count = 20,
                     window = 3,
                     vector_size = 300,
                     sample = 6e-5,
                     alpha = 0.03,
                     min_alpha = 0.0007,
                     negative = 20,
                     workers = cores-1)

model = Word2Vec.load('./output/gensim-model-All')

vectorized_docs = vectorize(corpus_clean, model = model)
len(vectorized_docs), len(vectorized_docs[0])

(35305, 300)

In [8]:
# Open jsonData file as data
with open('./data/jsonDataFinal.json') as json_file:
    data = json.load(json_file)

# Create df with data
df = pd.DataFrame(data)

# Replace np.nan values for empty list where no decision 
df['Decision:'] = [ [] if isinstance(x, float) else x for x in df['Decision:']]

# Original texts and labels of the decisions tokenized and ready to train a w2v
decisions = df['Decision:'].tolist()
decisions_labels = df['Decision label:'].tolist()


In [9]:


vectorized_decisions = vectorize(decisions, model = model)
len(vectorized_decisions), len(vectorized_decisions[0])

(35305, 300)

In [11]:


docs = df['Decision:'].tolist()
decision_labels = df['Decision label:'].tolist()


clustering, cluster_labels = mbkmeans_clusters(
	X = vectorized_decisions,
    k = 2,
    mb = 500,
    print_silhouette_values = True,
)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in corpus_clean],
    "cluster": cluster_labels,
    "label": decision_labels
})

For n_clusters = 2
Silhouette coefficient: 0.42
Inertia:423048.1420920639
Silhouette values:
    Cluster 1: Size:9190 | Avg:0.60 | Min:0.03 | Max: 0.78
    Cluster 0: Size:26115 | Avg:0.35 | Min:-0.13 | Max: 0.53


In [12]:
df_clusters.head(100)

,text,tokens,cluster,label
0,"[notice, of, decision, directions, the, decisi...",pic iac fh ck v upper tribunal immigration asy...,1,Accepted
1,"[in, light, of, my, conclusions, on, that, poi...",utijr jr upper tribunal immigration asylum cha...,1,Rejected
2,"[notice, of, decision, the, decision, of, the,...",pic upper tribunal immigration asylum chamber ...,0,Rejected
3,"[notice, of, decision, the, decision, of, the,...",pic upper tribunal immigration asylum chamber ...,0,Accepted
4,"[decision, the, decision, of, tribunal, judge,...",pic upper tribunal immigration asylum chamber ...,0,Accepted
...,...,...,...,...
95,"[decision, the, judge, materially, erred, in, ...",pic upper tribunal immigration asylum chamber ...,0,Accepted
96,"[decision, the, judge, materially, erred, in, ...",pic upper tribunal immigration asylum chamber ...,0,Accepted
97,"[from, the, point, at, which, the, applicant, ...",upper tribunal pic jr field house breams build...,1,Neutral
98,[],pic case jr v upper tribunal immigration asylu...,1,NaN


In [13]:
print("Most representative terms per cluster (based on centroids):")
for i in range(50):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")


Most representative terms per cluster (based on centroids):
Cluster 0: decision tribunal appeal tier aside dismiss remade determination ftt remake 
Cluster 1: decision tribunal appeal tier ftt accordingly aside judge determination error 


IndexError: index 2 is out of bounds for axis 0 with size 2

# 3. AVERAGED WORD2VEC ON TEXT OF THE RULING DESCRIBING THE DECISION

The strategy consists on training a word2vec model using the part of the text describing the judges decision from each the 35305 rulings. The results are averaged per document. Clustering algorithms are used to identify types of documents and their topics. There should be three types of documents: "Accepted", "Rejected" and "Salomonic/cannot tell".

In [30]:
# Number of processing cores
model_judgements = Word2Vec.load('./output/gensim-model-corpus_judgements')


In [31]:
clean_decisions = [x for x in decisions if x != None]

vectorized_decisions = vectorize(decisions, model = model_judgements)

len(vectorized_decisions), len(vectorized_decisions[0])

(35305, 300)

In [32]:

clustering, cluster_labels = mbkmeans_clusters(
	X = vectorized_decisions,
    k = 3,
    mb = 500,
    print_silhouette_values = True,
)
df_clusters = pd.DataFrame({
    "text": decisions,
    "tokens": [" ".join(text) for text in decisions],
    "cluster": cluster_labels,
    "label": decisions_labels
})

For n_clusters = 3
Silhouette coefficient: 0.34
Inertia:120377.10873388828
Silhouette values:
    Cluster 2: Size:9810 | Avg:0.55 | Min:0.02 | Max: 0.73
    Cluster 1: Size:4238 | Avg:0.40 | Min:-0.02 | Max: 0.57
    Cluster 0: Size:21257 | Avg:0.23 | Min:-0.12 | Max: 0.43


In [39]:
print("Most representative terms per cluster (based on centroids):")
for i in range(3):
    tokens_per_cluster = ""
    most_representative = model_judgements.wv.most_similar(positive = [clustering.cluster_centers_[i]], topn = 15)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Most representative terms per cluster (based on centroids):
Cluster 0: decision the aside set of law appeal error tribunal notice making i point involved remake 
Cluster 1: tribunal procedure direction anonymity rule unless until directs upper rules regarding otherwise the of court 
Cluster 2: the of i to aside decision appeal set it is law tribunal be that tier 


In [40]:
df_clusters

,text,tokens,cluster,label
0,"[notice, of, decision, directions, the, decisi...",notice of decision directions the decision of ...,2,Accepted
1,"[in, light, of, my, conclusions, on, that, poi...",in light of my conclusions on that point littl...,2,Rejected
2,"[notice, of, decision, the, decision, of, the,...",notice of decision the decision of the tribuna...,1,Rejected
3,"[notice, of, decision, the, decision, of, the,...",notice of decision the decision of the tribuna...,0,Accepted
4,"[decision, the, decision, of, tribunal, judge,...",decision the decision of tribunal judge malcol...,0,Accepted
...,...,...,...,...
35300,"[conclusions, a, northern, cyprus, is, not, ca...",conclusions a northern cyprus is not capable o...,2,Accepted
35301,"[for, the, reasons, we, have, given, we, dismi...",for the reasons we have given we dismiss this ...,2,Rejected
35302,"[our, conclusions, on, the, general, issues, r...",our conclusions on the general issues relating...,2,Rejected
35303,"[conclusions, for, each, of, the, main, reason...",conclusions for each of the main reason a and ...,2,Accepted


doc2vec

In [19]:
from gensim.models.doc2vec import Doc2Vec

model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

NameError: name 'train_tagged' is not defined